In [17]:
#-*-coding:utf-8 -*-

In [5]:
import torch
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import WarmupLinearSchedule as get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

import csv

In [6]:
n_devices = torch.cuda.device_count()
print(n_devices)

1


In [8]:
for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

NVIDIA GeForce GTX 1060


In [10]:
import asyncio
import websockets

async def echo(websocket, path):
    async for message in websocket:
        print(message)
        # message를 분석하여 주피터 노트북으로 값을 전달하는 코드 작성
        #분석
        websocket.send_text(f"Message text was: {data}")

async def main():
    async with websockets.serve(echo, "localhost", 8765):
        await asyncio.Future()  # 서버가 계속 실행되도록 대기

# 현재 실행 중인 이벤트 루프 가져오기
loop = asyncio.get_event_loop()

# 이벤트 루프에서 비동기 코드 실행
loop.create_task(main())

ModuleNotFoundError: No module named 'websockets'